In [361]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [362]:
df = pd.read_csv('Lezioni.csv').drop(columns='Unnamed: 0')

In [363]:
pd.DataFrame(df[(df['Index'] >= 269) & (df['Index'] <= 294)]['Professoressa'].value_counts().head(25))

,count
Professoressa,
Giusy,3
Rita,3
Giada 2,2
Miriam,2
Mariacristina,2
Francesca 3,1
Ileana,1
Erika,1
Mirela,1


In [364]:
df[df['Professoressa'] == 'Rita']

,Index,Data,Ora,Professoressa,Livello,Lezione,Giorno della settimana,Mese,Anno
44,45,2023-06-27,8,Rita,C1,C119,Martedì,Giugno,2023
125,126,2023-09-25,7,Rita,C1,C137,Lunedì,Settembre,2023
126,127,2023-09-26,7,Rita,B2,B221,Martedì,Settembre,2023
129,130,2023-10-02,7,Rita,B2,B235,Lunedì,Ottobre,2023
140,141,2023-10-10,7,Rita,C1,C136,Martedì,Ottobre,2023
156,157,2023-10-30,11,Rita,B2,B221,Lunedì,Ottobre,2023
160,161,2023-11-06,7,Rita,B2,B218,Lunedì,Novembre,2023
163,164,2023-11-08,7,Rita,B2,B240,Mercoledì,Novembre,2023
192,193,2024-01-09,7,Rita,B1,B118,Martedì,Gennaio,2024
207,208,2024-02-22,10,Rita,B1,B112,Giovedì,Febbraio,2024


In [365]:
df['MESE_ANNO'] = df['Mese'].astype(str) + '_' + df['Anno'].astype(str)
df.head()

,Index,Data,Ora,Professoressa,Livello,Lezione,Giorno della settimana,Mese,Anno,MESE_ANNO
0,1,2023-05-22,8,Virginia,B1,B132,Lunedì,Maggio,2023,Maggio_2023
1,2,2023-05-22,11,Graziana,B1,B135,Lunedì,Maggio,2023,Maggio_2023
2,3,2023-05-23,10,Francesca,C1,C123,Martedì,Maggio,2023,Maggio_2023
3,4,2023-05-23,11,Sara,B1,B143,Martedì,Maggio,2023,Maggio_2023
4,5,2023-05-24,10,Alice,B1,B141,Mercoledì,Maggio,2023,Maggio_2023


In [366]:
df_ascending_false = df.sort_values(by='Data', ascending=False)
df_ascending_false.head()#.to_clipboard()

,Index,Data,Ora,Professoressa,Livello,Lezione,Giorno della settimana,Mese,Anno,MESE_ANNO
332,333,2024-08-26,10,Miriam,C1,C124,Lunedì,Agosto,2024,Agosto_2024
331,332,2024-08-26,8,Alessandra,C1,C116,Lunedì,Agosto,2024,Agosto_2024
330,331,2024-08-23,9,Gaia,B2,B213,Venerdì,Agosto,2024,Agosto_2024
329,330,2024-08-21,7,Marta,B2,B221,Mercoledì,Agosto,2024,Agosto_2024
328,329,2024-08-20,11,Cecilia,C1,C132,Martedì,Agosto,2024,Agosto_2024


In [367]:
MESE_ANNO_ORDINE =['Maggio_2023', 'Giugno_2023', 'Luglio_2023', 'Settembre_2023', 'Ottobre_2023',
                   'Novembre_2023', 'Dicembre_2023', 'Gennaio_2024', 'Febbraio_2024', 'Marzo_2024',
                   'Aprile_2024', 'Maggio_2024', 'Giugno_2024', 'Luglio_2024', 'Agosto_2024']

In [368]:
dic = df_ascending_false.groupby('MESE_ANNO')['Index'].count()
dizionario_ordinato = {chiave: dic[chiave] for chiave in MESE_ANNO_ORDINE if chiave in dic}

In [375]:
df.head()

,Index,Data,Ora,Professoressa,Livello,Lezione,Giorno della settimana,Mese,Anno,MESE_ANNO
0,1,2023-05-22,8,Virginia,B1,B132,Lunedì,Maggio,2023,Maggio_2023
1,2,2023-05-22,11,Graziana,B1,B135,Lunedì,Maggio,2023,Maggio_2023
2,3,2023-05-23,10,Francesca,C1,C123,Martedì,Maggio,2023,Maggio_2023
3,4,2023-05-23,11,Sara,B1,B143,Martedì,Maggio,2023,Maggio_2023
4,5,2023-05-24,10,Alice,B1,B141,Mercoledì,Maggio,2023,Maggio_2023


In [369]:
# Crea il grafico a linee
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(dizionario_ordinato.keys()), y=list(dizionario_ordinato.values()), mode='lines+markers', name='Linea'))

# Aggiungi titolo e etichette agli assi
fig.update_layout(
    title='Quantità di Livelli per MESE_ANNO',
    xaxis_title='MESE_ANNO',
    yaxis_title='Quantità',
    legend_title='Livelli',
    template='plotly_white'
)

# Mostra il grafico
fig.show()

In [370]:
#df.groupby(['MESE_ANNO', 'Livello']).size().unstack().fillna(0).reset_index()

In [371]:
"""#df = df.groupby(['MESE_ANNO', 'Livello']).size().unstack().fillna(0).reset_index()

# Creazione del grafico
fig = go.Figure()

for column in df.columns[1:]:  # Ignora la colonna 'MESE_ANNO'
    fig.add_trace(go.Scatter(x=df['MESE_ANNO'], y=df[column], mode='lines+markers', name=column))

# Impostazioni del layout
fig.update_layout(title='Quantità di Livelli per MESE_ANNO',
                  xaxis_title='MESE_ANNO',
                  yaxis_title='Quantità',
                  legend_title='Livelli',
                  template='plotly_white')

# Mostra il grafico
fig.show()"""

"#df = df.groupby(['MESE_ANNO', 'Livello']).size().unstack().fillna(0).reset_index()\n\n# Creazione del grafico\nfig = go.Figure()\n\nfor column in df.columns[1:]:  # Ignora la colonna 'MESE_ANNO'\n    fig.add_trace(go.Scatter(x=df['MESE_ANNO'], y=df[column], mode='lines+markers', name=column))\n\n# Impostazioni del layout\nfig.update_layout(title='Quantità di Livelli per MESE_ANNO',\n                  xaxis_title='MESE_ANNO',\n                  yaxis_title='Quantità',\n                  legend_title='Livelli',\n                  template='plotly_white')\n\n# Mostra il grafico\nfig.show()"

In [372]:
"""def line_graphic(list_dic, eixo_X, eixo_y, colors, names_line):
    fig = go.Figure()

    for dic, color, name_line in zip(list_dic, colors, names_line):
        fig.add_trace(go.Scatter(x=list(dic.keys()), y=list(dic.values()), mode='lines', name=name_line, line=dict(color=color)))
        fig.update_layout(plot_bgcolor="white", xaxis_title=eixo_y, yaxis_title=eixo_X)

    fig.show() # lightgrey turquoise # deepskyblue violet lightseagreen lightpink"""

'def line_graphic(list_dic, eixo_X, eixo_y, colors, names_line):\n    fig = go.Figure()\n\n    for dic, color, name_line in zip(list_dic, colors, names_line):\n        fig.add_trace(go.Scatter(x=list(dic.keys()), y=list(dic.values()), mode=\'lines\', name=name_line, line=dict(color=color)))\n        fig.update_layout(plot_bgcolor="white", xaxis_title=eixo_y, yaxis_title=eixo_X)\n\n    fig.show() # lightgrey turquoise # deepskyblue violet lightseagreen lightpink'

In [373]:
ORDINE = df['MESE_ANNO'].to_list()
print(ORDINE)

['Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Maggio_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Giugno_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2023', 'Luglio_2

In [374]:
TOTALE = {chiave: df.groupby('MESE_ANNO')['Data'].count().sort_values(ascending=False).to_dict()[chiave] for chiave in ORDINE}
line_graphic([TOTALE], '', '', ['turquoise'], ['C1'])